In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from itertools import product

Series = pd.Series
DataFrame = pd.DataFrame

In [3]:
# Data cleaning

# This data set is available at https://www.kaggle.com/datasets/sobhanmoosavi/us-accidents
# It is the March23 version.
doc = pd.read_csv("US_Accidents_March23.csv")

# There are several columns that we're not investigating e.g., the length of the road affected by the accident.
doc = doc.reindex(columns=["ID", "Start_Time", "City", "Weather_Condition"])

# Some weather conditions are missing, so we drop those.
doc = doc.dropna(subset=["Weather_Condition"])
doc.head()

,ID,Start_Time,City,Weather_Condition
0,A-1,2016-02-08 05:46:00,Dayton,Light Rain
1,A-2,2016-02-08 06:07:59,Reynoldsburg,Light Rain
2,A-3,2016-02-08 06:49:27,Williamsburg,Overcast
3,A-4,2016-02-08 07:23:34,Dayton,Mostly Cloudy
4,A-5,2016-02-08 07:39:07,Dayton,Mostly Cloudy


In [4]:
# We're going to restrict attention to the 10 largest cities in the United States
cities = ["New York", "Los Angeles", "Chicago", "Houston", "Phoenix", "Philadelphia", "San Antonio", "San Diego", "Dallas", "Austin", "Jacksonville"]
doc = doc[doc["City"].isin(cities)]
doc.head()

,ID,Start_Time,City,Weather_Condition
42866,A-42867,2016-06-21 10:46:30,Los Angeles,Clear
42867,A-42868,2016-06-21 10:49:21,Los Angeles,Clear
42881,A-42882,2016-06-21 10:51:45,Los Angeles,Clear
42883,A-42884,2016-06-21 10:56:24,Los Angeles,Clear
42886,A-42887,2016-06-21 10:57:39,San Diego,Scattered Clouds


In [5]:
# We break down the hours into 24 discrete time slots. This is probably a mistake given our eventual circular encoding.
def previous_hour_mark(date_str):
    date, time = date_str.split()
    time = time[:2]
    return date + " " + time

doc["Start_Time"] = doc["Start_Time"].apply(previous_hour_mark)
doc.head()

,ID,Start_Time,City,Weather_Condition
42866,A-42867,2016-06-21 10,Los Angeles,Clear
42867,A-42868,2016-06-21 10,Los Angeles,Clear
42881,A-42882,2016-06-21 10,Los Angeles,Clear
42883,A-42884,2016-06-21 10,Los Angeles,Clear
42886,A-42887,2016-06-21 10,San Diego,Scattered Clouds


In [6]:
# To handle the case of a single (city, hour) pair having multiple different weather conditions, we simply take the most common.
def most_common(df, column="Weather_Condition"):
    return Series([df[column].mode()[0], len(df[column])])

aggregated = doc.groupby(["Start_Time", "City"]).apply(most_common)
aggregated.head()

0  1
Start_Time    City                         
2016-03-22 19 Los Angeles          Clear  1
2016-03-22 20 Los Angeles          Clear  2
              San Diego            Clear  1
2016-03-22 22 Los Angeles          Clear  2
2016-03-23 00 San Diego    Partly Cloudy  1

In [7]:
# The data from the previous aggregation is a bit cumbersome to feed to the regression objects of sklearn. The issue is the hierarchical indexing.
# We flatten it out here. There could probably be some optimizations made here using vectorized calculations. The data set is not so large
# that this takes a huge amount of time though.
data = {
    "Crashes": [aggregated.iloc[i][1] for i in range(len(aggregated))],
    "City": [aggregated.iloc[i].name[1] for i in range(len(aggregated))],
    "Hour": [aggregated.iloc[i].name[0][-2:] for i in range(len(aggregated))],
    "Weather": [aggregated.iloc[i][0] for i in range(len(aggregated))],
}
flat = DataFrame(data)
flat

,Crashes,City,Hour,Weather
0,1,Los Angeles,19,Clear
1,2,Los Angeles,20,Clear
2,1,San Diego,20,Clear
3,2,Los Angeles,22,Clear
4,1,San Diego,00,Partly Cloudy
...,...,...,...,...
270359,3,Chicago,21,Cloudy
270360,1,Phoenix,21,Mostly Cloudy
270361,1,Chicago,22,Mostly Cloudy
270362,2,Dallas,22,Fair


In [8]:
# There are close to 80 different weather conditions, some that have very few incidents attached to them. We collapse these into an "Other" category.
# This category ends up with about 8000 out of 270_000 entries, or about 3%.
def collapser(category, to_keep, other_str="Other"):
    s = set(to_keep)
    if category in s:
        return category
    else:
        return other_str
    
to_keep = ["Fair", "Mostly Cloudy", "Clear", "Cloudy", "Partly Cloudy", "Overcast", "Light Rain", "Scattered Clouds", "Haze", "Fog", "Rain", "Light Snow", "Light Drizzle", "Heavy Rain"]

flat["Weather"] = flat["Weather"].apply(lambda x : collapser(x, to_keep, other_str="Other"))

# We assert that we haven't lost any events.
assert flat["Weather"].value_counts().sum() == 270364

In [9]:
# The objects can be quite large in memory, and we can just work with the flat object from here on out.
# We delete these to reclaim some memory.
del doc
del aggregated

In [16]:
# Make our test-train split.
train, test = train_test_split(flat.copy(), 
                               shuffle=True,
                               random_state=614,
                               test_size=.2)

In [10]:
# This is a function for adding the one-hot encoding of the city and weather. We delete a column out of each one to avoid colinearity.
def one_hot_encode(df, in_place=False):
    if not in_place:
        df = df.copy()
    df.loc[:, pd.get_dummies(df["City"]).columns] = pd.get_dummies(df["City"], dtype=int).copy()
    del df["Austin"]
    del df["City"]
    df.loc[:, pd.get_dummies(df["Weather"]).columns] = pd.get_dummies(df["Weather"], dtype=int).copy()
    del df["Other"]
    del df["Weather"]
    return df

# This transforms the hour column into two that are cos(2*pi*hour/24) and sin(2*pi*hour/24). This preserves the circular structure
# of the hour marks. 

def circular_encode_hour(df, in_place=False):
    if not in_place:
        df = df.copy()
    df["HourSin"] = df["Hour"]
    df["HourCos"] = df["Hour"]
    df["HourSin"] = df["HourSin"].apply(lambda x : np.sin(2*np.pi*int(x) / 24))
    df["HourCos"] = df["HourCos"].apply(lambda x : np.cos(2*np.pi*int(x) / 24))
    del df["Hour"]
    return df

In [13]:
# Function for adding in the interaction terms for our categorical variables.
# Pandas does give some performance warnings, but it runs quickly enough for this dataset.
import warnings
encoded_cities = ["Chicago", "Dallas", "Houston", "Jacksonville", "Los Angeles", "New York", "Philadelphia", "San Antonio", "Phoenix"]
encoded_weather = ["Fair", "Mostly Cloudy", "Clear", "Cloudy", "Partly Cloudy", "Overcast", "Light Rain", "Scattered Clouds", "Haze", "Fog", "Rain", "Light Snow", "Light Drizzle", "Heavy Rain"]
times = ["HourCos", "HourSin"]
cities_weather_prods = list(product(encoded_cities, encoded_weather))
cities_times_prods = list(product(encoded_cities, times))
weather_times_product = list(product(encoded_weather, times))
triple_product = list(product(encoded_cities, encoded_weather, times))
def add_in_twofold_interaction_terms(df):
    warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
    df = df.copy()
    for city, weather in cities_weather_prods:
        df[f"{city} {weather}"] = df[city]*df[weather]
    for city, time in cities_times_prods:
        df[f"{city} {time}"] = df[city]*df[time]
    for weather, time in weather_times_product:
        df[f"{weather} {time}"] = df[weather]*df[time]
    df = df.copy()
    warnings.simplefilter(action='default', category=pd.errors.PerformanceWarning)
    return df

In [14]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error as mse

from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet

In [17]:
# We use a 5-fold cross validation
kfold = KFold(n_splits=5, shuffle=True)

In [18]:
# We have 7 models:
# 1. The baseline which is just the average over each (city,hour) pair.
# 2. Linear regression minimizing the residual sum of squares without interaction terms.
# 3. Ridge regression without interaction.
# 4. Elastic net regression without interaction.
# 5-7. The previous regressions with interaction terms.

mses = np.zeros((7,5))
for i, (train_index, test_index) in enumerate(kfold.split(train)):
    train_cv = train.iloc[train_index]
    test_cv = train.iloc[test_index]

    
    # baseline model
    grouped = train_cv.groupby(["City", "Hour"])
    grouped_crashes = grouped["Crashes"]
    grouped_crashes_mean = grouped_crashes.agg("mean")
    baseline = Series(grouped_crashes_mean[test_cv.iloc[i]["City"], test_cv.iloc[i]["Hour"]] for i in range(len(test_cv)))
    
    # Setting up the data without interaction terms
    one_hot = one_hot_encode(train_cv)
    circular_one_hot = circular_encode_hour(one_hot)
    no_interaction_args = (circular_one_hot.iloc[:, 1:].values, train_cv["Crashes"].values)
    no_int_test = circular_encode_hour(one_hot_encode(test_cv)).iloc[:, 1:].values
    
    # Linear regression for residual sum of squares without interaction
    no_interaction_lr = LinearRegression(copy_X=True)
    no_interaction_lr.fit(*no_interaction_args)
    no_int_pred = no_interaction_lr.predict(no_int_test)
    
    # Ridge regression without interaction
    no_int_rr = Ridge(copy_X=True)
    no_int_rr.fit(*no_interaction_args)
    no_int_rr_pred = no_int_rr.predict(no_int_test)
    
    # Elastic net without interaction
    no_int_en = ElasticNet(copy_X=True)
    no_int_en.fit(*no_interaction_args)
    no_int_en_pred = no_int_en.predict(no_int_test)
    
    # Interaction setup
    interaction_terms = add_in_twofold_interaction_terms(circular_one_hot)
    interaction_args = (interaction_terms.iloc[:, 1:].values, train_cv["Crashes"].values)
    int_test = add_in_twofold_interaction_terms(circular_encode_hour(one_hot_encode(test_cv))).iloc[:, 1:].values
    
    # Linear regression with twofold interaction
    # Not passing the positive=True parameter sometimes resulted in a negative coefficient that
    # was very large (~10**20) in absolute value.
    twofold_interaction_lr = LinearRegression(copy_X=True, positive=True)
    twofold_interaction_lr.fit(*interaction_args)
    twofold_int_pred = twofold_interaction_lr.predict(int_test)
    
    # Ridge regression without interaction
    int_rr = Ridge(copy_X=True)
    int_rr.fit(*interaction_args)
    int_rr_pred = int_rr.predict(int_test)
    
    # Elastic net without interaction
    int_en = ElasticNet(copy_X=True)
    int_en.fit(*interaction_args)
    int_en_pred = int_en.predict(int_test)
    
    mses[0,i] = mse(test_cv["Crashes"].values, baseline)
    mses[1,i] = mse(test_cv["Crashes"].values, no_int_pred)
    mses[2,i] = mse(test_cv["Crashes"].values, no_int_rr_pred)
    mses[3,i] = mse(test_cv["Crashes"].values, no_int_en_pred)
    mses[4,i] = mse(test_cv["Crashes"].values, twofold_int_pred)
    mses[5,i] = mse(test_cv["Crashes"].values, int_rr_pred)
    mses[6,i] = mse(test_cv["Crashes"].values, int_en_pred)
mses

array([[6.94004178, 7.02272696, 7.05489417, 7.10133208, 7.39265402],
       [7.35569775, 7.48015731, 7.51399227, 7.5777256 , 7.93770107],
       [7.35572641, 7.48016795, 7.51400142, 7.57769656, 7.9376663 ],
       [8.39405473, 8.49467992, 8.52751956, 8.60043697, 8.94605655],
       [7.60124345, 7.65844433, 7.6934709 , 7.77364321, 8.13339989],
       [7.11475217, 7.24295612, 7.26974304, 7.32090655, 7.68496401],
       [8.39405473, 8.49467992, 8.52751956, 8.60043697, 8.94605655]])

In [22]:
# None of our models beat the baseline model though the ridge regression with
# interaction was closest.
for i in range(7):
    print(np.round(np.mean(mses, axis=1)[i],4))

7.1023
7.5731
7.5731
8.5925
7.772
7.3267
8.5925
